In [1]:
#gcloud auth application-default login

In [2]:
import csv
import datetime

import vertexai
from vertexai.generative_models import GenerationConfig, GenerativeModel, Part, Tool
from vertexai.preview.generative_models import grounding
from vertexai.preview import caching

PROJECT_ID = "TODO"

## JSON OUTPUT

In [3]:
prompt_template = """
You are a name matching expert.
Task: Determine if these names likely refer to the same person.

Name 1: {name1}
Name 2: {name2}

Answer with a likeliness score from 0 to 100.
"""
model = GenerativeModel(
    model_name="gemini-1.5-flash",
    generation_config={"response_mime_type": "application/json"},
  )
    
def generate(name1, name2):
  vertexai.init(project=PROJECT_ID, location="us-central1")
  prompt = prompt_template.format(name1=name1, name2=name2)
  
  response = model.generate_content(prompt)
  return response

csv_file_path = "names.csv"

results = []

with open(csv_file_path, 'r', newline='') as csvfile:
    reader = csv.reader(csvfile)
    next(reader, None) 
    for row in reader:
        name1, name2 = row  
        result = generate(name1, name2).text
        results.append(result) 

In [4]:
print(results)

['{"score": "20"}\n', '{"score": 60}\n', '{"likelihood_score": "90"}\n', '{"likelihood_score": 80}\n', '{"likelihood_score": 95}\n', '{"score": 95}\n\n', '{"likelihood_score": 95}\n', '{"likelihood_score": 95}\n', '{"likelihood_score": "90"}\n', '{"score": 90}\n\n', '{"score": 90}\n', '{"likelihood_score": 95}\n', '{"likelihood_score": 100}\n', '{"score": 90}\n', '{"score": 90}\n\n', '{"score": "70"}\n', '{"likelihood_score": "90"}\n']


In [5]:
response_schema = {
  "type": "object",
  "properties": {
    "firstName": {
      "type": "string",
      "description": "The first name"
    },
    "secondName": {
      "type": "string",
      "description": "The second name"
    },
    "score": {
      "type": "integer",
      "description": "The likelihood score"
    },
    "explanation": {
      "type": "string",
      "description": "The explanation of the score"
    }
  },
  "required": [
    "firstName",
    "secondName",
    "score",
    "explanation"
  ]
}

In [6]:
response = model.generate_content(
    prompt_template.format(name1="Mathias", name2="mathius"),
    generation_config=GenerationConfig(
        response_mime_type="application/json", response_schema=response_schema
    ),
)

In [7]:
print(response.text)

{"explanation": "The names 'Mathias' and 'mathius' are very similar. The only difference is the capitalization. The slight spelling variation is common, and it is very likely that these names refer to the same person.", "firstName": "Mathias", "score": 95, "secondName": "mathius"} 


## Grounding in enterprise data

In [8]:
# Grounding in Enterprise data

generation_config = {
    "max_output_tokens": 8192,
    "temperature": 1,
    "top_p": 0.95,
}

tools = [
    Tool.from_retrieval(
        retrieval=grounding.Retrieval(
            source=grounding.VertexAISearch(datastore="projects/vertexai-demo-420409/locations/global/collections/default_collection/dataStores/alphabet-investors-files_1716910033705"),
        )
    ),
]

def generate_enterprise(prompt):
    vertexai.init(project=PROJECT_ID, location="us-central1")
    model = GenerativeModel(
        "gemini-1.5-flash-001",
        tools=tools,
    )
    response = model.generate_content(
        [prompt],
        generation_config=generation_config,
    )

    print(response.text, end="")
    return response

generate_enterprise("""What was Alphabet R&D cost in 2009?""")

Alphabet's R&D cost in 2009 was $2,843,027,000. 


candidates {
  content {
    role: "model"
    parts {
      text: "Alphabet\'s R&D cost in 2009 was $2,843,027,000. \n"
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
    probability_score: 0.2265625
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.1904296875
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
    probability_score: 0.1259765625
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.0888671875
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
    probability_score: 0.259765625
    severity: HARM_SEVERITY_LOW
    severity_score: 0.2890625
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
    probability_score: 0.1982421875
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.1875
  }
  grounding_metadata {
    retrieval_queries: "What was Alph

In [9]:
generate_enterprise("""What was Alphabet R&D cost in 2024?""")

The provided sources do not contain information on Alphabet's R&D cost in 2024. 


candidates {
  content {
    role: "model"
    parts {
      text: "The provided sources do not contain information on Alphabet\'s R&D cost in 2024. \n"
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
    probability_score: 0.23046875
    severity: HARM_SEVERITY_LOW
    severity_score: 0.26171875
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
    probability_score: 0.2177734375
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.12255859375
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
    probability_score: 0.310546875
    severity: HARM_SEVERITY_LOW
    severity_score: 0.2890625
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
    probability_score: 0.2109375
    severity: HARM_SEVERITY_LOW
    severity_score: 0.22265625
  }
  grounding_metadata {
    retrieval_quer

In [10]:
# Grounding in google search

tools = [
    Tool.from_google_search_retrieval(
        google_search_retrieval=grounding.GoogleSearchRetrieval()
    ),
]

def generate_with_search(prompt):
    vertexai.init(project="vertexai-demo-420409", location="us-central1")
    model = GenerativeModel(
        "gemini-1.5-flash-001",
        tools=tools,
    )
    response = model.generate_content(
        [prompt],
        generation_config=generation_config,
    )

    print(response, end="")
    
generate_with_search("""What was Alphabet R&D cost in 2024?""")

candidates {
  content {
    role: "model"
    parts {
      text: "Alphabet\'s R&D expenses for the twelve months ending June 30, 2024, were $47.134 billion, a 10.65% increase year-over-year. This means that Alphabet spent $47.134 billion on research and development in the year leading up to June 30, 2024. \n\nIt\'s important to note that this figure includes R&D spending across all of Alphabet\'s subsidiaries, including Google,  and not just Google\'s R&D costs.  Also, R&D expenses are constantly changing, so this figure is specific to the twelve months ending June 30, 2024, and does not reflect the full year\'s expenditures. \n"
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
    probability_score: 0.05419921875
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.1416015625
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
    probability_score: 0.0727539062

## Context Caching

In [12]:

vertexai.init(project=PROJECT_ID, location="us-central1")

system_instruction = """
You are an expert researcher. You always stick to the facts in the sources provided, and never make up new facts.
Now look at these research papers, and answer the following questions.
"""

contents = [
    Part.from_uri(
        "gs://cloud-samples-data/generative-ai/pdf/2312.11805v3.pdf",
        mime_type="application/pdf",
    ),
    Part.from_uri(
        "gs://cloud-samples-data/generative-ai/pdf/2403.05530.pdf",
        mime_type="application/pdf",
    ),
]

cached_content = caching.CachedContent.create(
    model_name="gemini-1.5-pro-001",
    system_instruction=system_instruction,
    contents=contents,
    ttl=datetime.timedelta(minutes=60),
    display_name="example-cache",
)

cache_id = cached_content.name
print(cache_id)

3503976431954690048


In [14]:
from vertexai.preview.generative_models import GenerativeModel

vertexai.init(project=PROJECT_ID, location="us-central1")

cached_content = caching.CachedContent(cached_content_name=cache_id)

model = GenerativeModel.from_cached_content(cached_content=cached_content)

response = model.generate_content("What are the papers about?")

print(response.text)

The first paper is about Gemini, a family of multimodal models developed at Google. Gemini is trained to understand and reason across text, images, audio and video. This paper introduces the first version, Gemini 1.0, which comes in three sizes: Ultra, Pro, and Nano. The authors evaluate the models on a comprehensive suite of benchmarks and demonstrate their state-of-the-art performance.

The second paper focuses on Gemini 1.5 Pro, a new addition to the Gemini family. This model is distinguished by its exceptional long-context understanding capability, enabling it to process information from up to 10 million tokens. This paper details Gemini 1.5 Pro’s architecture, training infrastructure, and evaluation across various tasks including multimodal long-context understanding, in-context language learning, long-document and video QA, and long-context ASR. The authors highlight the model's performance advancements while emphasizing responsible AI practices throughout its development and dep